In [ ]:
!pip install python-docx
!pip install nltk

In [2]:
import docx
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
import pandas as pd

In [3]:
# open
whitepaper_dict = pickle.load(open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/whitepaper_dict.pkl', 'rb'))

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install utils

## Data Cleanup

#### Remove Short Sentences (sentence with less than 5 tokens) and repeated symbols, spaces



In [ ]:
whitepaper_dict_cleaned = {}
counter = 0
for key, value in whitepaper_dict.items():

    document_cleaned = []
    for sentence in value[1]:
        sentence = sentence.replace("::", "")
        sentence = sentence.replace("--", "")
        sentence = sentence.replace("- ", "")
        sentence = sentence.replace("  ", "")
        
        if len(word_tokenize(sentence)) > 5:
          document_cleaned.append(sentence)

    whitepaper_dict_cleaned[key] = document_cleaned

In [ ]:
#save
pickle.dump(whitepaper_dict_cleaned, open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/whitepaper_dict_cleaned.pkl', 'wb'))

In [6]:
#read
whitepaper_dict_cleaned = pickle.load(open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/whitepaper_dict_cleaned.pkl', 'rb'))

In [ ]:
len(whitepaper_dict_cleaned.keys())


342

## SBERT Model and Cosine Similarity

#### Embeddings for all documents using whitepaper clean dict

In [ ]:
from sentence_transformers import SentenceTransformer, util

def sentence_embeddings(sentence):
  embedder = SentenceTransformer('paraphrase-distilroberta-base-v2')
  return embedder.encode(sentence, convert_to_tensor=True)


In [ ]:
sentence_embeddings_dict = {} 

for key, value in whitepaper_dict_cleaned.items():

  output = sentence_embeddings(value)
  sentence_embeddings_dict[key] = output


In [ ]:
#save
pickle.dump(sentence_embeddings_dict, open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/sentence_embeddings_dict_updated.pkl', 'wb'))

In [7]:
#read
sentence_embeddings_dict_updated = pickle.load(open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/sentence_embeddings_dict_updated.pkl', 'rb'))

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import time
import pandas as pd


def main(sentences_file_name, sentences_file, queries_file_name, queries_file, threshold=0.8, exact_match=True):

  start = time.time()

  corpus = sentences_file
  queries = queries_file

  corpus_embeddings = sentence_embeddings_dict_20[sentences_file_name]
  query_embeddings = sentence_embeddings_dict_20[queries_file_name]

  # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
  n = 10 
  top_k = 5

  data = []
  counter = 0
  for query in queries:

    top_k_list = get_query_top_k(query,query_embeddings[counter, :], corpus, corpus_embeddings, max_n = n, top_k = top_k, min_p=threshold)
    data.extend(top_k_list)
    counter += 1

  df = pd.DataFrame(data)
  end = time.time()
  e = int(end - start)
  # print(end-start)
  # print('Time elapsed is: {:02d}:{:02d}:{:02d}'.format(e // 3600, (e % 3600 // 60), e % 60))

  return df


In [ ]:
len(sentence_embeddings_dict_updated.keys())

342

In [ ]:
from sentence_transformers import util
import torch
import pandas as pandas

def similarity(query_embeddings,docs_embeddings,max_n=10, top_k=5):

  cos_scores = util.pytorch_cos_sim(query_embeddings, docs_embeddings)[0]
  cos_scores = cos_scores.cpu()

  top_results = torch.topk(cos_scores, k=max_n) 

  return zip(top_results[0], top_results[1])

def get_query_top_k(query, query_embeddings, docs, docs_embeddings, max_n=10, top_k=5, min_p=0.7, exact_match=True):

  count=0
  top_k_list = []

  for score, idx in similarity(query_embeddings, docs_embeddings, max_n=max_n):
    score = score.item() #score is tensor
    if count<top_k and ((score>min_p and exact_match) or (score<=0.99 and score>min_p)): 
      count=count+1 
      top_k_list.append({"query":query,"sentence":docs[idx],"score":score}) 
  return top_k_list

##### W/ Threshold 0.8 & whitepaper_dict_cleaned



In [ ]:
comparison_dict_08_cleaned_all = {}
#sentence_embeddings_dict
key_list = list(sentence_embeddings_dict_updated.keys())

for i in range(len(key_list)-1):
  key1 = key_list[i]
  try:
  # if key1 != 'Umbrella.docx': 

    for key2 in key_list[i+1:]:

      output = main(key1, whitepaper_dict_cleaned[key1], key2, whitepaper_dict_cleaned[key2], threshold=0.8, exact_match=True)
      comparison_dict_08_cleaned_all[key1+'@'+key2] = output

  except:
    pass

In [ ]:
#save
pickle.dump(comparison_dict_08_cleaned_all, open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/comparison_dict_08_cleaned_all.pkl', 'wb'))

In [8]:
#read
comparison_dict_08_cleaned_all = pickle.load(open('/content/drive/MyDrive/Plagiarism/Whitepapers_txt/comparison_dict_08_cleaned_all.pkl', 'rb'))

In [ ]:
import pandas as pd
df1 = pd.DataFrame(columns = ['key', 'query', 'source', 'score'])
counter = 0
for key, value in comparison_dict_08_cleaned_all.items(): 
  if len(value) > 0:
    for i in value.values:
      score = i[2]
      query = i[1]
      source = i[0]

      # some_list.append([key, ])
      df1.loc[counter, :] = [key, query, source, score]
      counter += 1


##### df with sent len > 5 and no filter

In [ ]:
#save df1 in csv
df1.to_csv('/content/drive/MyDrive/2021_Indep_Study_Plagiarism/Whitepapers_txt/SBERTcomparison_dict_08_cleaned_all.csv')  --- > I overwrote it so its been replaced with the final 


In [ ]:
#previous results -- sentence > 5 toks
df1['key'].value_counts()

CelerNetwork-Whitepaper.docx@Celer.docx              913
shyft-network-inc-whitepaper_v4.1.docx@Shyft.docx    301
Origo.docx@Cortex.docx                               206
Sport_and_Leisure.docx@AllSports.docx                192
Tokenomy.docx@Origo.docx                             141
                                                    ... 
PLATINCOIN.docx@Lympo.docx                             1
Refereum.docx@LINA.docx                                1
Elastos.docx@Fusion.docx                               1
Elastos.docx@HTMLCOIN.docx                             1
Karma.docx@MedicCoin.docx                              1
Name: key, Length: 4536, dtype: int64

In [ ]:
#read

df = pd.read_csv('/content/drive/MyDrive/2021_Indep_Study_Plagiarism/Whitepapers_txt/SBERTcomparison_dict_08_cleaned_all.csv')

##### disclaimer list extracted from the similarity matches

In [11]:
# below list of pair matches contained nothing but legal disclaimers

pair_list = ['Origo.docx@Cortex.docx',
 'Tokenomy.docx@Origo.docx',
 'Tokenomy.docx@Covesting.docx',
 'ULTRAIN.docx@Celer.docx',
 'Tokenomy.docx@Cortex.docx',
 'Origo.docx@Covesting.docx',
 'Swarm.docx@MediBloc.docx',
 'UNetwork.docx@Egretia.docx',
 'Tokenomy.docx@Levolution.docx']

In [12]:
disclaimer_list = []

for pair in pair_list:
  disclaimer_list.extend(set(list((df[df['key'] == pair][['query', 'source']].values.flatten()))))

In [13]:
len(disclaimer_list)

1059

In [15]:
print(df.shape)
df.head()

(12512, 5)


,Unnamed: 0,key,query,source,score
0,0,ZVCHAIN.docx@Unibright.docx,the contract will eventually be written in the...,"after its creation, the smart contract will be...",0.828529
1,1,ZVCHAIN.docx@ShipChain.docx,the contract will eventually be written in the...,"that contract will be completed on delivery, w...",0.802803
2,2,ZVCHAIN.docx@QuarkChain.docx,we contend that in the context of the blockcha...,"as aforementioned, decentralization also gives...",0.859681
3,3,ZVCHAIN.docx@QuarkChain.docx,privacy-preserving computing and security fram...,"as aforementioned, decentralization also gives...",0.804344
4,4,ZVCHAIN.docx@Acala_Whitepaper.docx,regardless if it is public chain or private ch...,the significance of cross-chain communication ...,0.800443


##### remove disclaimer hits and sentence length less than 20 tokens

In [22]:
df_new = pd.DataFrame(columns = df.columns)
i = 0
for index, row in df.iterrows():
  if (row['query'] in disclaimer_list) or (row['source'] in disclaimer_list) or (len(row['source'].split(' ')) < 20 ) or (len(row['query'].split(' ')) < 20 ):
    continue

  else:
    df_new.loc[i,:] = row.values
    i += 1


In [23]:
len(df_new)

2310

##### remove duplicate whitepapers

In [26]:
df_new = df_new[df_new['key'] != 'CelerNetwork-Whitepaper.docx@Celer.docx']
df_new = df_new[df_new['key'] != 'shyft-network-inc-whitepaper_v4.1.docx@Shyft.docx']

In [27]:
df_new['key'].value_counts()

Sport_and_Leisure.docx@AllSports.docx    124
PRIZM.docx@Nxt.docx                       81
Levolution.docx@INS.docx                  17
RealTract.docx@Constellation.docx         15
MakerDAO.docx@Dai.docx                    15
                                        ... 
AcuteAngleCould.docx@MediBloc.docx         1
Litex.docx@Alphacat.docx                   1
Primas.docx@Dimecoin.docx                  1
StakeNet.docx@QASH.docx                    1
QuarkChain.docx@Eximchain.docx             1
Name: key, Length: 1016, dtype: int64

In [29]:
df_new.to_csv('/content/drive/MyDrive/2021_Indep_Study_Plagiarism/Whitepapers_txt/SBERTcomparison_dict_final.csv')